In [5]:
import pandas as pd
import numpy as np
import pprint
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

class ID3DecisionTree:
    """
    Decision tree using ID3
    """
    def __init__(self):
        pass

    def get_entropy(self, df):
        # TODO: Calculate the entropy(s)
        class_values, class_counts = np.unique(df['play'], return_counts=True)
        probabilities = class_counts / len(df)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def split_table(self, df, attr, value):
        return df[df[attr] == value].reset_index(drop=True)

    def fit(self, df):

        tree={}
        # compute entropy(s)
        entropy_s = self.get_entropy(df)

        # List of all attrs except the class
        attrs = df.keys()[:-1]

        max_gain = 0
        max_gain_attr = None
        # for all attr, find the attr with max gain
        for attr in attrs: #Outlook,Temparature,Humidity,Wind
            values = df[attr].unique()
            # TODO: Calculate the entropies of unique values of attr
            # Example: attr=Outlook has entropies = [0.971, 0.971, 0]
            entropies = []

            # TODO: Calculate the probabilities of unique values of attr
            # Example: attr=Outlook has probabilities = [5/14, 5/14, 0]
            probabilities = []

            for value in values:  #Sunny,Rain,Overcast (Outlook), same for others
              subset=df[df[attr]==value]
              subset_entropy=self.get_entropy(subset)
              entropies.append(subset_entropy)
              probabilities.append(len(subset)/len(df))


            # Calculate the average information entropy
            average_info_entropy = 0
            for probability, entropy in zip(probabilities, entropies):
                average_info_entropy += probability * entropy

            # TODO: Calculate attr gain
            attr_gain = entropy_s - average_info_entropy


            # TODO: Update the max_gain_attr
            if attr_gain > max_gain:
                max_gain = attr_gain
                max_gain_attr = attr

        tree[max_gain_attr] = {}
        # Split the df based on the values of max_gain_attr
        values = df[max_gain_attr].unique()
        for value in values:
            new_df = self.split_table(df, max_gain_attr, value)
            class_values, class_counts = np.unique(new_df['play'],return_counts=True)

            # If it is a pure class, then this is the leaf node
            # else divide the new_df further
            if len(class_counts)==1:
                tree[max_gain_attr][value] = class_values[0]
            else:
                tree[max_gain_attr][value] = self.fit(new_df)

        return tree

    def predict(self, example, tree, default=None):
        attribute = next(iter(tree))
        if example[attribute] in tree[attribute].keys():
            subtree = tree[attribute][example[attribute]]
            if isinstance(subtree, dict):
                return self.predict(example, subtree)
            else:
                return subtree
        else:
            return default

    def evaluate(self, tree, df):
        # TODO: Complete the evaluate method
        correct_predictions = 0
        for index, row in df.iterrows():
            example = row.drop('play').to_dict()
            prediction = self.predict(example, tree)
            if prediction == row['play']:
                correct_predictions += 1
        accuracy = correct_predictions / len(df) * 100
        return accuracy

# TODO: Complete the class for decision tree using CART
class CARTDecisionTree:
    def __init__(self):
        self.tree = None

    def fit(self, df):
        self.tree = self.build_tree(df, 'play')
        return self.tree

    def gini_impurity(self, labels):
        # Calculate Gini impurity for a set of labels
        if len(labels) == 0:
            return 0.0

        # Count occurrences of each class
        class_counts = labels.value_counts()

        # Calculate Gini impurity
        total_samples = len(labels)
        impurity = 1.0
        for count in class_counts:
            proportion = count / total_samples
            impurity -= proportion ** 2

        return impurity

    def find_best_split(self, df, target_column):
        # Find the best attribute to split on based on Gini impurity
        best_split_attr = None
        best_split_value = None
        min_gini = float('inf')

        for attribute in df.columns:
            if attribute == target_column:
                continue

            # Get unique values of the attribute
            values = df[attribute].unique()

            for value in values:
                # Split the dataset
                left_subset = df[df[attribute] <= value]
                right_subset = df[df[attribute] > value]

                # Calculate Gini impurity for the split
                left_impurity = self.gini_impurity(left_subset[target_column])
                right_impurity = self.gini_impurity(right_subset[target_column])

                # Weighted sum of impurities
                weighted_impurity = (len(left_subset) / len(df)) * left_impurity \
                                    + (len(right_subset) / len(df)) * right_impurity

                # Update best split if the current split is better
                if weighted_impurity < min_gini:
                    min_gini = weighted_impurity
                    best_split_attr = attribute
                    best_split_value = value

        return best_split_attr, best_split_value
    def build_tree(self, df, target_column):
        # Base case: If all examples have the same label, create a leaf node
          if len(df[target_column].unique()) == 1:
            return {'class': df[target_column].iloc[0]}

        # Find the best attribute and value to split on
          best_split_attr, best_split_value = self.find_best_split(df, target_column)

        # If no split improves purity, create a leaf node
          if best_split_attr is None:
            return {'class': df[target_column].mode().iloc[0]}

        # Split the dataset
          left_subset = df[df[best_split_attr] <= best_split_value]
          right_subset = df[df[best_split_attr] > best_split_value]

        # Recursively build left and right subtrees
          left_subtree = self.build_tree(left_subset, target_column)
          right_subtree = self.build_tree(right_subset, target_column)

        # Return the current node with split information and subtrees
          return {'attribute': best_split_attr,
                  'value': best_split_value,
                  'left': left_subtree,
                  'right': right_subtree}


    def predict(self, example, tree):
        # Predict the class for a single example
        # Recursive function to predict the class for a single example
        if 'class' in tree:
            return tree['class']
        else:
            attr_value = example[tree['attribute']]
            if attr_value <= tree['value']:
                return self.predict(example, tree['left'])
            else:
                return self.predict(example, tree['right'])

    def evaluate(self, tree, df, target_column='play'):
        # Evaluate the accuracy of the trained tree on the given DataFrame
        examples = df.drop(target_column, axis=1)
        true_labels = df[target_column]

        predictions = [self.predict(example, tree) for _, example in examples.iterrows()]

        # Calculate accuracy
        correct_predictions = sum(pred == true_label for pred, true_label in zip(predictions, true_labels))
        accuracy = correct_predictions / len(true_labels) * 100
        return accuracy

# Read the dataset
# data_path = '/content/drive/MyDrive/Data Mining Lab/finaldataset.csv'
data_path = '/content/drive/MyDrive/MlLab/play_tennis.csv'


df = pd.read_csv(data_path)
# print(df)

# TODO: Shuffle the df randomly
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
# print(df)

# TODO: Split the df into train_df and test_df using 80:20 ratio
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
# print(train_df)
# print(test_df)

# Train the model
model = ID3DecisionTree()

tree = model.fit(train_df)

# Visualize the decision tree
print('the decision tree for ID3 algorithm:\n')
pprint.pprint(tree)


# Predict an example
x = {'Outlook': 'Rain', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak'}
y_pred = model.predict(x, tree)
print("#ID3 algorithm: Output class:", y_pred)

# Evaluate the model
acc = model.evaluate(tree, test_df)
print("#ID3 algorithm: Accuracy: {:.3f}".format(acc))



#Train model
model = CARTDecisionTree()

# Fit the tree on the training data
# Assuming 'target' is the name of the target variable column
tree = model.fit(train_df)
print('\n The decision tree for CART:\n')
# Visualize the decision tree
pprint.pprint(tree)

# Predict an example
x = {'Outlook': 'Rain', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak'}
y_pred = model.predict(x, tree)
print("#CART algorithm: Output class:", y_pred)

# Evaluate the model
acc = model.evaluate(tree, test_df, target_column='play')
print("#CART algorithm: Accuracy: {:.3f}".format(acc))



the decision tree for ID3 algorithm:

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Temperature': {'Cool': 'Yes',
                                       'Hot': 'No',
                                       'Mild': 'No'}}}}
#ID3 algorithm: Output class: Yes
#ID3 algorithm: Accuracy: 66.667

 The decision tree for CART:

{'attribute': 'Outlook',
 'left': {'class': 'Yes'},
 'right': {'attribute': 'Wind',
           'left': {'class': 'No'},
           'right': {'attribute': 'Outlook',
                     'left': {'class': 'Yes'},
                     'right': {'attribute': 'Temperature',
                               'left': {'class': 'Yes'},
                               'right': {'class': 'No'},
                               'value': 'Cool'},
                     'value': 'Rain'},
           'value': 'Strong'},
 'value': 'Overcast'}
#CART algorithm: Output class: Yes
#CART algorithm: Accuracy: 66.667


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
